In [11]:
!pip install transformers faiss-cpu datasets sentence-transformers

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline

In [38]:
documents = [
    "Riyadh is the capital of Saudi Arabia.",
    "Cairo is the capital of Egypt.",
    "Paris is the capital of France.",
    "Tokyo is the capital of Japan.",
    "Berlin is the capital of Germany.",
    "Rome is the capital of Italy."
]

In [39]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = embedder.encode(documents, convert_to_numpy=True, batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

In [41]:
qa_model = pipeline("text2text-generation", model="google/flan-t5-base")


Device set to use cpu


In [44]:
def rag_pipeline(query):
    query_embedding = embedder.encode([query])
    D, I = index.search(query_embedding, k=3)

    retrieved_docs = [documents[i] for i in I[0]]

    context = "\n".join(retrieved_docs)
    prompt = f"{context}\nAnswer the question:\n{query}"

    result = qa_model(prompt, max_new_tokens=50)[0]['generated_text']
    return result



In [47]:
# My Test
query = "What is the capital of saudi arabia?"
answer = rag_pipeline(query)
print("Answer:", answer)

Answer: Riyadh
